## Capstone Project 1: Semantic Search
### Name: Santosh Shirol
### Batch: MTech AI02

In [3]:
#Import required library
import pandas as pd
import numpy as np

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm


In [4]:
#Load products and category
df_products = pd.read_json("products_scraper.json")
df_products

,product_description,category
0,Antenna booster for Mercedes-Benz E-Class 210 ...,amplifiers
1,Amplifier for Audi A6 Sedan C6 (05.2004 - 03.2...,amplifiers
2,Amplifier for Audi A6 Avant C6 (03.2005 - 08.2...,amplifiers
3,Antenna booster for Volkswagen Passat V Varian...,amplifiers
4,Amplifier for Volkswagen Phaeton Sedan (04.200...,amplifiers
...,...,...
5971,Cassette player for Seat Ibiza I Hatchback (06...,cassette_and_cd_players
5972,CD player for Renault Laguna II Grandtour (03....,cassette_and_cd_players
5973,CD player and climate control panel for Mazda ...,cassette_and_cd_players
5974,CD player for Renault Laguna II Grandtour (03....,cassette_and_cd_players


In [5]:
#Take few records for experiment
#df_products = df_products.sample(frac=1).reset_index(drop=True)
#df_products = df_products[1:20]

In [6]:
df_products

,product_description,category
0,Antenna booster for Mercedes-Benz E-Class 210 ...,amplifiers
1,Amplifier for Audi A6 Sedan C6 (05.2004 - 03.2...,amplifiers
2,Amplifier for Audi A6 Avant C6 (03.2005 - 08.2...,amplifiers
3,Antenna booster for Volkswagen Passat V Varian...,amplifiers
4,Amplifier for Volkswagen Phaeton Sedan (04.200...,amplifiers
...,...,...
5971,Cassette player for Seat Ibiza I Hatchback (06...,cassette_and_cd_players
5972,CD player for Renault Laguna II Grandtour (03....,cassette_and_cd_players
5973,CD player and climate control panel for Mazda ...,cassette_and_cd_players
5974,CD player for Renault Laguna II Grandtour (03....,cassette_and_cd_players


In [27]:
df_products[df_products['category'] == 'amplifiers']

,product_description,category
0,Antenna booster for Mercedes-Benz E-Class 210 ...,amplifiers
1,Amplifier for Audi A6 Sedan C6 (05.2004 - 03.2...,amplifiers
2,Amplifier for Audi A6 Avant C6 (03.2005 - 08.2...,amplifiers
3,Antenna booster for Volkswagen Passat V Varian...,amplifiers
4,Amplifier for Volkswagen Phaeton Sedan (04.200...,amplifiers
5,"Antenna booster for Rover 75 1.8, 120 hp, seda...",amplifiers
6,"Amplifier for Honda Accord VI 1.8, 136 hp, sed...",amplifiers
7,Amplifier for Volvo XC90 I SUV (06.2002 - 01.2...,amplifiers
8,Amplifier for Audi A4 Sedan B7 (11.2004 - 06.2...,amplifiers
9,Antenna booster for Mercedes-Benz S-Class W220...,amplifiers


In [28]:
df_products['category'].value_counts()

amplifiers                                                     36
starters                                                       36
distributors                                                   36
external_boot_lid_handles                                      36
door_handles_and_locks_doors                                   36
                                                               ..
heating_blowers                                                36
light_switches                                                 36
ignition_key_security_cylinder_lock_and_ignition_switch_key    36
interior_courtesy_light                                        36
cassette_and_cd_players                                        36
Name: category, Length: 166, dtype: int64

In [7]:
import string


# nltk library Stop words list
stop_words = {'hasnt', 'for', 'ma', 'up', 'should', 'which', 'now', 'her', 'so', 'these', 'don', 'll', 'youd', 'against', 'doing', 'my', 'mightnt', 'him', 'but', 'is', 'dont', 'shouldve', 'arent', 'then', 'during', 't', 'above', 'once', 'shouldn', 'we', 'themselves', 're', 'was', 'needn', 'herself', 'has', 'be', 'as', 'from', 'until', 'between', 'his', 'hadn', 'mustn', 'under', 'too', 'through', 'mustnt', 'can', 'ours', 'theirs', 'me', 'you', 'shouldnt', 'she', 'over', 'or', 'isn', 'in', 'your', 'haven', 'ourselves', 'again', 'further', 'when', 'no', 'o', 'he', 'what', 'himself', 'all', 'after', 'will', 'been', 'have', 'not', 'being', 'other', 'having', 'few', 'both', 'than', 'that', 'it', 'some', 'about', 'their', 'whom', 'its', 'are', 'had', 'out', 'into', 'where', 've', 'our', 'the', 'youve', 'them', 'nor', 'just', 'while', 'am', 'down', 'd', 'm', 'of', 'doesnt', 'why', 'hers', 'shant', 'wasn', 'havent', 'hadnt', 'aren', 'wouldnt', 'who', 'by', 'here', 'shan', 'didn', 'such', 'own', 'below', 'neednt', 'same', 'if', 'off', 'myself', 'a', 'each', 'this', 'thatll', 'how', 'youre', 'does', 'yourselves', 'do', 'very', 'isnt', 'any', 'wont', 'werent', 'those', 'because', 'yourself', 'y', 'won', 'did', 'at', 'couldnt', 'more', 'its', 'there', 'with', 'on', 'itself', 'only', 'an', 'before', 'mightn', 'yours', 'ain', 'they', 'wasnt', 'were', 'doesn', 'shes', 'weren', 'most', 'didnt', 'hasn', 'youll', 'wouldn', 'to', 's', 'i', 'couldn', 'and'}


# make all text lowercase
def text_lowercase(text):
    return text.lower()


# remove stopwords
def remove_stopwords(text):
    words = text.split()
    text = [i for i in words if not i in stop_words]
    text = ' '.join(text)
    return text


# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


# preprocess text string
def preprocessing(text):
    text = text_lowercase(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    return text


In [8]:
# SBERT model 
model = SentenceTransformer('all-MiniLM-L6-v2')
def get_vector(product_description,category=""):
    productandcategory = product_description+category
    product_details = preprocessing(productandcategory)
    products  = [productandcategory]
    products_embeddings = model.encode(products)
    _ = list(products_embeddings.flatten())
    encod_np_array = np.array(_)
    products_embeddings_list = encod_np_array.tolist()
    #print(len(products_embeddings_list))
    return products_embeddings_list

In [38]:

tqdm.pandas()
df_products["product_vector"] = df_products.progress_apply(lambda x: get_vector(x.product_description, x.category), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 5976/5976 [01:29<00:00, 66.80it/s]


In [18]:
#df_products=df_products.rename(columns={'products_vector':'product_vector'})

In [39]:
products_data = df_products.to_dict("records")



In [40]:
len(products_data)

5976

In [33]:
#Establish connection with ElasticSearch
client = Elasticsearch(['http://localhost:9200'], http_auth=("elastic", "NzAQmqmHd7-ouB+fo9-4"))


C:\Users\I331215\AppData\Local\Temp\ipykernel_27064\2225361553.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  client = Elasticsearch(['http://localhost:9200'], http_auth=("elastic", "NzAQmqmHd7-ouB+fo9-4"))


In [34]:
client.ping()

2023-02-02 20:40:49.986 HEAD http://localhost:9200/ [status:200 duration:0.082s]


True

In [11]:
# Fetch index information
client.indices.get_mapping(index="myproducts")

ObjectApiResponse({'myproducts': {'mappings': {'properties': {'category': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'product_description': {'type': 'text'}, 'product_vector': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine'}}}}})

In [41]:
#index values to elastic search - Push product details with generated vector 
for product in products_data:
    client.index(index="myproducts",body=product)

C:\Users\I331215\AppData\Local\Temp\ipykernel_39764\1599758539.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.index(index="myproducts",body=product)


In [19]:
#index values to elastic search - this without vector
for product in products_data:
    client.index(index="myproductswithoutvector",body=product)

C:\Users\I331215\AppData\Local\Temp\ipykernel_39764\338138148.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.index(index="myproductswithoutvector",body=product)


In [18]:
products_data

[{'product_description': 'Antenna booster for Mercedes-Benz E-Class 210 (W/S) 2.0, 136 hp, sedan, 5 doors, 1998 ',
  'category': 'amplifiers'},
 {'product_description': 'Amplifier for Audi A6 Sedan C6 (05.2004 - 03.2011) ',
  'category': 'amplifiers'},
 {'product_description': 'Amplifier for Audi A6 Avant C6 (03.2005 - 08.2011) ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Volkswagen Passat V Variant B6 (08.2005 - 11.2011), № 3C9 035 552 ',
  'category': 'amplifiers'},
 {'product_description': 'Amplifier for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 3D0 035 465A ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Rover 75 1.8, 120 hp, sedan, 1999 ',
  'category': 'amplifiers'},
 {'product_description': 'Amplifier for Honda Accord VI 1.8, 136 hp, sedan, 2001 № Bose / 39190-S1A-00 ',
  'category': 'amplifiers'},
 {'product_description': 'Amplifier for Volvo XC90 I SUV (06.2002 - 01.2015), № 30732825 ',
  'category': 'amplifi

In [33]:
search_string = input("Enter the Input Query ")
token_vector = get_vector(search_string,search_string)
token_vector

Enter the Input Query antena
384


[-0.004093733616173267,
 0.02357134409248829,
 -0.06676733493804932,
 0.004734553396701813,
 -0.04035312309861183,
 -0.04426214471459389,
 0.052442289888858795,
 0.04255903512239456,
 -0.027889035642147064,
 0.024130670353770256,
 0.028040101751685143,
 -0.01495275180786848,
 -0.10206850618124008,
 -0.004777778871357441,
 -0.03451210632920265,
 0.023740168660879135,
 0.005177771206945181,
 -0.011500303633511066,
 0.06042799726128578,
 0.051443230360746384,
 -0.008246995508670807,
 -0.009661983698606491,
 -0.02543332614004612,
 0.004635734483599663,
 -0.015372026711702347,
 0.12404852360486984,
 -0.022231431677937508,
 0.025433745235204697,
 -0.0076817418448626995,
 -0.07162125408649445,
 -0.05134682357311249,
 0.07172346115112305,
 0.07972430437803268,
 -0.012163601815700531,
 -0.05468189716339111,
 -0.06712175160646439,
 -0.06828362494707108,
 0.011421361938118935,
 0.030866630375385284,
 0.05984216183423996,
 -0.020189717411994934,
 0.023323120549321175,
 -0.0530722476541996,
 -0.014

## Semantic Search using SBERT

In [65]:
search_string = input("Enter the Input Query ")
token_vector = get_vector(search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

Enter the Input Query fuel injector


[{'product_description': 'Gasoline fuel injector for Ford Mondeo Mk II 1.6, 90 hp, sedan, 5 doors, 2000 '},
 {'product_description': 'Gasoline fuel injector for Hyundai Coupe 1.6 16V, 114 hp, 1998 '},
 {'product_description': 'Gasoline fuel injector for Rover 200 Hatchback II (11.1995 - 03.2000) 214 i, 75 hp '},
 {'product_description': 'Gasoline fuel injector for Opel Tigra Coupe (07.1994 - 12.2000) 1.4 16V, 90 hp '},
 {'product_description': 'Gasoline fuel injector for Opel Tigra 1.4 16V, 90 hp, 1995 '},
 {'product_description': 'Gasoline fuel injector for Mercedes-Benz A-Class Hatchback W168 (07.1997 - 08.2004) A 160 (168.033, 168.133), 102 hp '},
 {'product_description': 'Gasoline fuel injector for Lancia Delta 2.0 16V, 139 hp, hatchback, 5 doors, 1994 '},
 {'product_description': 'Gasoline fuel injector for Volvo S40/V40 1.8, 115 hp, sedan automatic, 1997 '},
 {'product_description': 'Gasoline fuel injector for Citroen Xsara Hatchback (04.1997 - 04.2005) 1.6 i, 88 hp, № 0280155794

In [12]:
search_string = "Antenna"
token_vector = get_vector(search_string,search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

[{'product_description': 'Antenna for Mercedes-Benz M-Class (W164) (07.2005 - ...), № A 251 820 22 75 '},
 {'product_description': 'Antenna for Volvo S70/V70 2.3 T5, 250 hp, station wagon automatic, 2000 № 9494817 '},
 {'product_description': 'Antenna booster for Subaru Legacy IV Wagon (09.2003 - 12.2009), № EF1203IC '},
 {'product_description': 'Keyless go antenna for Lexus IS III Sedan (04.2013 - ...) '},
 {'product_description': 'Keyless go antenna for Lexus IS III Sedan (04.2013 - ...) '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Keyless go antenna for Volkswagen Phaeton S

In [61]:
search_string = "sensors"
token_vector = get_vector(search_string,search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

[{'product_description': 'Temperature sensor for Ford Fiesta IV 1.8 D, 60 hp, hatchback, 1999 '},
 {'product_description': 'Camshaft sensor for Ford Mondeo I Turnier (01.1993 - 08.1996) '},
 {'product_description': 'MAP sensor for Opel Corsa D 1.4, 90 hp, 5 doors, 2007 № Bosch 0 261 230 101 '},
 {'product_description': 'Air temperature sensor for Volkswagen Phaeton 5.0 TDI 4motion, 313 hp automatic, 2003 № 4B0 907 659 '},
 {'product_description': 'Parktronic for Volkswagen Sharan Minivan I (05.1995 - 03.2010) '},
 {'product_description': 'Parktronic for Volkswagen Sharan Minivan I (05.1995 - 03.2010) '},
 {'product_description': 'Parktronic for Volkswagen Sharan Minivan I (05.1995 - 03.2010) '},
 {'product_description': 'Camshaft sensor for Ford Fiesta V Hatchback (11.2001 - 03.2010) '},
 {'product_description': 'Camshaft sensor for Renault Megane II Coupe-Cabriolet (09.2003 - 03.2010) '},
 {'product_description': 'Central locking sensor for Hyundai Tucson 2.0 CRDi 4WD, 113 hp, suv, 20

In [64]:
search_string = "gearbox"
token_vector = get_vector(search_string,search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

[{'product_description': 'Transmission module for Porsche Boxster 986 2.7, 220 hp, cabrio automatic, 2001 № Bosch 0 260 002 670 '},
 {'product_description': 'Automatic gearbox for Subaru Forester SUV I (03.1997 - 09.2002) 2.0 AWD, 122 hp, automatic, № TZ103ZR1AA-L2 '},
 {'product_description': 'Automatic gearbox for Land Rover Range Rover III SUV (03.2002 - 08.2012) 4.4 4x4, 286 hp, automatic, № 7508713 '},
 {'product_description': 'Gearbox actuator for Smart City-Coupe 450 (07.1998 - 01.2004) 0.6 (S1CLB1, 450.331, 450.336), 45 hp '},
 {'product_description': 'Gearbox actuator for Mercedes-Benz M-Class W163 4.3, 272 hp, suv, 5 doors automatic, 2000 '},
 {'product_description': 'Automatic gearbox for Volkswagen Passat II Sedan B3, B4 (02.1988 - 12.1997) 2.0, 115 hp, automatic '},
 {'product_description': 'Automatic gearbox for Mercedes-Benz M-Class SUV (W164) (07.2005 - 12.2012) ML 320 CDI 4-matic (164.122), 224 hp, automatic, № 1642708200 '},
 {'product_description': 'Gearbox actuator 

In [67]:
search_string = "musical device"
token_vector = get_vector(search_string,search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

[{'product_description': 'CD player for Renault Scenic II Minivan (06.2003 - 07.2010) '},
 {'product_description': 'Cassette player for Ford Galaxy Minivan I (03.1995 - 05.2006) '},
 {'product_description': 'Cassette player for Seat Ibiza I Hatchback (06.1984 - 12.1993) '},
 {'product_description': 'CD player for Seat Altea Minivan (03.2004 - 12.2015) '},
 {'product_description': 'Cassette player for Volkswagen Passat IV Sedan B5.5 (10.2000 - 12.2005) '},
 {'product_description': 'Cassette player for Opel Frontera A (1991-1998) '},
 {'product_description': ' CD player for Opel Signum Hatchback (05.2003 - ...) '},
 {'product_description': 'CD player for Toyota Avensis (2009-2018) '},
 {'product_description': 'Loudspeakers for Mini Countryman (R60) (2010-2014) '},
 {'product_description': 'Loudspeakers for Citroen C4 Grand Picasso I (10.2006 - 12.2013) '}]

In [ ]:
search_string = "musical device"
token_vector = get_vector(search_string,search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

In [156]:
#booster 

In [30]:
df_products[df_products['product_description'].str.contains('booster ')]

,product_description,category
0,Antenna booster for Mercedes-Benz E-Class 210 ...,amplifiers
3,Antenna booster for Volkswagen Passat V Varian...,amplifiers
5,"Antenna booster for Rover 75 1.8, 120 hp, seda...",amplifiers
9,Antenna booster for Mercedes-Benz S-Class W220...,amplifiers
10,Antenna booster for Audi A6 Sedan C6 (05.2004 ...,amplifiers
13,Antenna booster for Land Rover Range Rover III...,amplifiers
14,Antenna booster for Volkswagen Phaeton Sedan (...,amplifiers
18,Antenna booster for Hyundai Santa Fe 2.4 16V 4...,amplifiers
19,Antenna booster for BMW 3 Series E46 Sedan (02...,amplifiers
21,Antenna booster for Audi A3 (8P1) (05.2003 - 0...,amplifiers


## Keyword based search 

In [73]:
data = client.search(
    index="myproductswithoutvector",
    body={
        "query": {
            "bool": {
                "must": {
                    "match_phrase": {
                        "product_description": "booster",
                    }
                }},
        }          
    }
)['hits']['hits']
[d['_source'] for d in data]

C:\Users\I331215\AppData\Local\Temp\ipykernel_39764\3205278906.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  data = client.search(


[{'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Rover 75 1.8, 120 hp, sedan, 1999 ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for BMW 3 Series E46 Sedan (02.1998 - 04.2005) ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Subaru Legacy IV Wagon (09.2003 - 12.2009), № EF1203IC ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Audi A3 (8P1) (05.2003 - 08.2012), № 8P0 035 225 ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for BMW X5 (E53) 3.0, 231 hp automatic, 2002 ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Audi A6 Sedan C6 (05.2004 - 03.2011), № 8E0 035 456 ',
  'category': 'amplifiers'},
 {'product_description': 'Antenna booster for Ford Kuga SUV I (02.2008 - 11.2012), № 8M5T-15K603-KA ',
  'category': 'ampl

In [32]:
client.search(
    index="myproductswithoutvector",
    body={
        "query": {
    "query_string": {
      "query": "booster",
      "default_field": "product_description"
    }
  }
        }
)['hits']['hits']

C:\Users\I331215\AppData\Local\Temp\ipykernel_39764\343480099.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.search(


[{'_index': 'myproductswithoutvector',
  '_id': 'W_VI8oUBbNhOkI4bJSjG',
  '_score': 6.6858864,
  '_source': {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 ',
   'category': 'amplifiers'}},
 {'_index': 'myproductswithoutvector',
  '_id': 'UvVI8oUBbNhOkI4bJSgt',
  '_score': 6.454427,
  '_source': {'product_description': 'Antenna booster for Rover 75 1.8, 120 hp, sedan, 1999 ',
   'category': 'amplifiers'}},
 {'_index': 'myproductswithoutvector',
  '_id': 'YPVI8oUBbNhOkI4bJigV',
  '_score': 6.454427,
  '_source': {'product_description': 'Antenna booster for BMW 3 Series E46 Sedan (02.1998 - 04.2005) ',
   'category': 'amplifiers'}},
 {'_index': 'myproductswithoutvector',
  '_id': 'ZvVI8oUBbNhOkI4bJihx',
  '_score': 6.454427,
  '_source': {'product_description': 'Antenna booster for Subaru Legacy IV Wagon (09.2003 - 12.2009), № EF1203IC ',
   'category': 'amplifiers'}},
 {'_index': 'myproductswithoutvector',
  '_id': 'YvVI8oUBbNhOkI4b

In [68]:
search_string = "musical device"
client.search(
    index="myproductswithoutvector",
    body={
        "query": {
    "query_string": {
      "query": search_string,
      "default_field": "product_description"
    }
  }
        }
)['hits']['hits']

C:\Users\I331215\AppData\Local\Temp\ipykernel_39764\3825321638.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.search(


[]

In [69]:
search_string = "CD"
client.search(
    index="myproductswithoutvector",
    body={
        "query": {
    "query_string": {
      "query": search_string,
      "default_field": "product_description"
    }
  }
        }
)['hits']['hits']

C:\Users\I331215\AppData\Local\Temp\ipykernel_39764\2112934765.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.search(


[{'_index': 'myproductswithoutvector',
  '_id': 'hfVJ8oUBbNhOkI4b6D-e',
  '_score': 5.464651,
  '_source': {'product_description': 'CD player for Toyota Avensis (2009-2018) ',
   'category': 'cassette_and_cd_players'}},
 {'_index': 'myproductswithoutvector',
  '_id': 'jvVJ8oUBbNhOkI4b6T9E',
  '_score': 5.464651,
  '_source': {'product_description': ' CD player for Opel Signum Hatchback (05.2003 - ...) ',
   'category': 'cassette_and_cd_players'}},
 {'_index': 'myproductswithoutvector',
  '_id': 'zvVJ8oUBbNhOkI4b2z5w',
  '_score': 5.2613964,
  '_source': {'product_description': 'CD changer for Peugeot 607 (9D, 9U) (01.2000 - ...) ',
   'category': 'cd_changer'}},
 {'_index': 'myproductswithoutvector',
  '_id': '0vVJ8oUBbNhOkI4b2z60',
  '_score': 5.2613964,
  '_source': {'product_description': 'CD changer for Volkswagen Phaeton Sedan (04.2002 - 03.2016) ',
   'category': 'cd_changer'}},
 {'_index': 'myproductswithoutvector',
  '_id': '3fVJ8oUBbNhOkI4b3D6D',
  '_score': 5.2613964,
  '_sou

In [14]:
search_string = "Antenna"
token_vector = get_vector(search_string,search_string)
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']
[d['_source'] for d in data]

[{'product_description': 'Antenna for Mercedes-Benz M-Class (W164) (07.2005 - ...), № A 251 820 22 75 '},
 {'product_description': 'Antenna for Volvo S70/V70 2.3 T5, 250 hp, station wagon automatic, 2000 № 9494817 '},
 {'product_description': 'Antenna booster for Subaru Legacy IV Wagon (09.2003 - 12.2009), № EF1203IC '},
 {'product_description': 'Keyless go antenna for Lexus IS III Sedan (04.2013 - ...) '},
 {'product_description': 'Keyless go antenna for Lexus IS III Sedan (04.2013 - ...) '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Keyless go antenna for Volkswagen Phaeton S

In [18]:
len(data)

10

In [29]:
# streamlit run C:\Users\I331215\Desktop\CaptstoneProject\code\app\app.py

In [31]:
t=[{'_index': 'myproducts', '_id': 'kfV58oUBbNhOkI4bmEnj', '_score': 0.63391185, '_source': {'product_description': 'Air conditioning panel for Mitsubishi Eclipse II Coupe (04.1994 - 04.1999) '}}, {'_index': 'myproducts', '_id': 'Z_V68oUBbNhOkI4b0Vbw', '_score': 0.6197499, '_source': {'product_description': 'Phone for Audi A8 (D3) 4.0 TDI Quattro, 275 hp automatic, 2003 '}}, {'_index': 'myproducts', '_id': 'afV68oUBbNhOkI4b0lYc', '_score': 0.6124212, '_source': {'product_description': 'Phone for Audi A8 (D3) 4.0 TDI Quattro, 275 hp, sedan, 5 doors automatic, 2007 '}}, {'_index': 'myproducts', '_id': 'kvV58oUBbNhOkI4bW0Y8', '_score': 0.6122865, '_source': {'product_description': 'Front wipers motor for Mitsubishi Eclipse II Coupe (04.1994 - 04.1999), coupe, position: front '}}, {'_index': 'myproducts', '_id': 'cPV68oUBbNhOkI4bm1RM', '_score': 0.6065009, '_source': {'product_description': 'GPS antenna for Lexus IS III Sedan (04.2013 - ...), № 86860-53070 '}}, {'_index': 'myproducts', '_id': 'A_V58oUBbNhOkI4beUhP', '_score': 0.6059206, '_source': {'product_description': 'Interior courtesy light for Volkswagen Touareg SUV I (10.2002 - 01.2013) '}}, {'_index': 'myproducts', '_id': '_PV68oUBbNhOkI4bkFN0', '_score': 0.6042881, '_source': {'product_description': 'Siren for Alfa Romeo 166 (936) (09.1998 - 06.2007), № 60674632 '}}, {'_index': 'myproducts', '_id': 'QPV58oUBbNhOkI4bVEbw', '_score': 0.60406387, '_source': {'product_description': 'AC heat air vent for Mitsubishi Eclipse II Coupe (04.1994 - 04.1999) '}}, {'_index': 'myproducts', '_id': 'ifV58oUBbNhOkI4bmEkv', '_score': 0.6028264, '_source': {'product_description': 'Air conditioning panel for Opel Zafira A Minivan (04.1999 - 06.2005) '}}, {'_index': 'myproducts', '_id': 'dfV48oUBbNhOkI4b40Ax', '_score': 0.6026716, '_source': {'product_description': 'Air conditioning hose for Nissan Murano I SUV (08.2003 - 09.2008) '}}]

In [32]:
t

[{'_index': 'myproducts',
  '_id': 'kfV58oUBbNhOkI4bmEnj',
  '_score': 0.63391185,
  '_source': {'product_description': 'Air conditioning panel for Mitsubishi Eclipse II Coupe (04.1994 - 04.1999) '}},
 {'_index': 'myproducts',
  '_id': 'Z_V68oUBbNhOkI4b0Vbw',
  '_score': 0.6197499,
  '_source': {'product_description': 'Phone for Audi A8 (D3) 4.0 TDI Quattro, 275 hp automatic, 2003 '}},
 {'_index': 'myproducts',
  '_id': 'afV68oUBbNhOkI4b0lYc',
  '_score': 0.6124212,
  '_source': {'product_description': 'Phone for Audi A8 (D3) 4.0 TDI Quattro, 275 hp, sedan, 5 doors automatic, 2007 '}},
 {'_index': 'myproducts',
  '_id': 'kvV58oUBbNhOkI4bW0Y8',
  '_score': 0.6122865,
  '_source': {'product_description': 'Front wipers motor for Mitsubishi Eclipse II Coupe (04.1994 - 04.1999), coupe, position: front '}},
 {'_index': 'myproducts',
  '_id': 'cPV68oUBbNhOkI4bm1RM',
  '_score': 0.6065009,
  '_source': {'product_description': 'GPS antenna for Lexus IS III Sedan (04.2013 - ...), № 86860-53070 '

In [35]:
res = {'product_description': 'Phone for Audi A8 (D3) 4.0 TDI Quattro, 275 hp, sedan, 5 doors automatic, 2007 '}

In [40]:
s = res
s['product_description'] = res['product_description']

In [41]:
s

{'product_description': 'Phone for Audi A8 (D3) 4.0 TDI Quattro, 275 hp, sedan, 5 doors automatic, 2007 '}